In [3]:
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

TASKS = ["lift", "can", "square", "transport"]
METHODS = ["kmeans", "hdbscan", "gmm", "spectral"]

def safe_load_image(path):
    """이미지 파일을 로드. 없으면 None 반환."""
    if path is None:
        return None
    if not os.path.exists(path):
        return None
    return mpimg.imread(path)

def make_5x4_grid(
    elbow_paths_by_task,
    vis_paths_by_method_and_task,
    out_path="kmeans_cluster_grid_5x4.png",
    dpi=200,
    figsize=(18, 20),
):
    """
    elbow_paths_by_task: dict[task] -> path (1행)
    vis_paths_by_method_and_task: dict[method][task] -> path (2~5행)
    """

    fig, axes = plt.subplots(nrows=5, ncols=4, figsize=figsize)

    # ---- Column titles (task)
    for c, task in enumerate(TASKS):
        axes[0, c].set_title(task, fontsize=14, pad=10)

    # ---- Row labels
    row_labels = ["Elbow (KMeans)", "KMeans", "HDBSCAN", "GMM", "Spectral"]
    for r in range(5):
        axes[r, 0].set_ylabel(row_labels[r], fontsize=12, rotation=90, labelpad=10)

    # ---- 1st row: elbow plots
    for c, task in enumerate(TASKS):
        ax = axes[0, c]
        img = safe_load_image(elbow_paths_by_task.get(task))
        if img is None:
            ax.text(0.5, 0.5, f"Missing\n{task} elbow", ha="center", va="center", fontsize=10)
        else:
            ax.imshow(img)
        ax.axis("off")

    # ---- Rows 2~5: method visualizations
    method_to_row = {"kmeans": 1, "hdbscan": 2, "gmm": 3, "spectral": 4}
    for method in METHODS:
        r = method_to_row[method]
        for c, task in enumerate(TASKS):
            ax = axes[r, c]
            img = safe_load_image(vis_paths_by_method_and_task.get(method, {}).get(task))
            if img is None:
                ax.text(0.5, 0.5, f"Missing\n{method}-{task}", ha="center", va="center", fontsize=10)
            else:
                ax.imshow(img)
            ax.axis("off")

    plt.tight_layout()
    fig.savefig(out_path, dpi=dpi, bbox_inches="tight")
    plt.close(fig)
    print(f"[Saved] {out_path}")

if __name__ == "__main__":
    # ✅ 당신이 저장해둔 파일 경로를 여기에 맞춰 넣기
    elbow_paths_by_task = {
        "lift":      "/home/sophia435256/workspace2/git/qc/plot/lift-mh-low_elbow.png",
        "can":       "/home/sophia435256/workspace2/git/qc/plot/can-mh-low_elbow.png",
        "square":    "/home/sophia435256/workspace2/git/qc/plot/square-mh-low_elbow.png",
        "transport": "/home/sophia435256/workspace2/git/qc/plot/transport-mh-low_elbow.png",
    }

    vis_paths_by_method_and_task = {
        "kmeans": {
            "lift":      "/home/sophia435256/workspace2/git/qc/plot/lift-mh-low_kmeans_centroids.png",
            "can":       "/home/sophia435256/workspace2/git/qc/plot/can-mh-low_kmeans_centroids.png",
            "square":    "/home/sophia435256/workspace2/git/qc/plot/square-mh-low_kmeans_centroids.png",
            "transport": "/home/sophia435256/workspace2/git/qc/plot/transport-mh-low_kmeans_centroids.png",
        },
        "hdbscan": {
            "lift":      "/home/sophia435256/workspace2/git/qc/plot/lift-mh-low_hdbscan_centroids.png",
            "can":       "/home/sophia435256/workspace2/git/qc/plot/can-mh-low_hdbscan_centroids.png",
            "square":    "/home/sophia435256/workspace2/git/qc/plot/square-mh-low_hdbscan_centroids.png",
            "transport": "/home/sophia435256/workspace2/git/qc/plot/transport-mh-low_hdbscan_centroids.png",
        },
        "gmm": {
            "lift":      "/home/sophia435256/workspace2/git/qc/plot/lift-mh-low_gmm_centroids.png",
            "can":       "/home/sophia435256/workspace2/git/qc/plot/can-mh-low_gmm_centroids.png",
            "square":    "/home/sophia435256/workspace2/git/qc/plot/square-mh-low_gmm_centroids.png",
            "transport": "/home/sophia435256/workspace2/git/qc/plot/transport-mh-low_gmm_centroids.png",
        },
        "spectral": {
            "lift":      "/home/sophia435256/workspace2/git/qc/plot/lift-mh-low_gmm_spectral_centroids.png",
            "can":       "/home/sophia435256/workspace2/git/qc/plot/can-mh-low_gmm_spectral_centroids.png",
            "square":    "/home/sophia435256/workspace2/git/qc/plot/square-mh-low_gmm_spectral_centroids.png",
            "transport": "/home/sophia435256/workspace2/git/qc/plot/transport-mh-low_gmm_spectral_centroids.png",
        },
    }

    make_5x4_grid(
        elbow_paths_by_task=elbow_paths_by_task,
        vis_paths_by_method_and_task=vis_paths_by_method_and_task,
        out_path="cluster_viz_5x4.png",
        dpi=250,
        figsize=(18, 22),
    )


[Saved] cluster_viz_5x4.png


In [4]:
!wandb login

wandb: Currently logged in as: sophia435256 (sophia435256-robros) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


In [6]:
import wandb
import pandas as pd

api = wandb.Api()

# 형식: "entity/project/run_id"
run = api.run("sophia435256-robros/qc/g9osubel")


In [9]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

api = wandb.Api(timeout=120)  # timeout 늘리기

RUN_PATHS = [
    "sophia435256-robros/qc/srm9ukq0",
    "sophia435256-robros/qc/jyox63ri",
    "sophia435256-robros/qc/81ptm9i2",
    "sophia435256-robros/qc/d1u9hv4o",
]

SUCCESS_KEY_CANDIDATES = [
    "eval/success",
]

def fetch_success_df(run_path: str) -> pd.DataFrame:
    run = api.run(run_path)

    # 어떤 success key가 있는지 자동으로 찾기 (summary/config로 힌트)
    possible_keys = set(list(run.summary.keys()) + list(run.config.keys()))
    success_key = None
    for k in SUCCESS_KEY_CANDIDATES:
        if k in possible_keys:
            success_key = k
            break

    # summary에 없더라도 history에만 있는 경우가 많아서, 후보들을 스캔해보며 찾기
    if success_key is None:
        # 먼저 작은 샘플로 후보 키들을 시도
        for k in SUCCESS_KEY_CANDIDATES:
            rows = list(run.scan_history(keys=["_step", k], page_size=500))
            if len(rows) > 0 and any((k in r and r[k] is not None) for r in rows):
                success_key = k
                break

    if success_key is None:
        raise KeyError(
            f"[{run_path}] success key를 찾지 못했습니다. "
            f"후보: {SUCCESS_KEY_CANDIDATES}\n"
            f"Run summary keys 예시: {list(run.summary.keys())[:50]}"
        )

    rows = []
    for r in run.scan_history(keys=["_step", success_key], page_size=5000):
        step = r.get("_step")
        val = r.get(success_key)
        if step is not None and val is not None:
            rows.append({"step": step, "success": float(val)})

    df = pd.DataFrame(rows).sort_values("step")
    df["run"] = run.name  # 범례는 run name으로
    return df

# 4개 run 합치기
dfs = []
for rp in RUN_PATHS:
    dfs.append(fetch_success_df(rp))

all_df = pd.concat(dfs, ignore_index=True)

# plot
plt.figure(figsize=(10, 5))
for run_name, g in all_df.groupby("run"):
    plt.plot(g["step"], g["success"], label=run_name)

plt.xlabel("step")
plt.ylabel("eval/success")
plt.title("Eval Success Comparison (4 runs)")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()


KeyboardInterrupt: 

In [ ]:
import wandb
    api = wandb.Api()
    run = api.run("/sophia435256-robros/qc/runs/d1u9hv4o")

print(run.history())